import pandas as pd

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import requests


rest_api_key = 'c09c78655acb25593e9c6f07b0f42f90'
key_commer = 'bG%2BZ%2BmDdiTVy%2Faq2OLF%2FKCockUZQnuHoXbUTjrFJYWbe5ZtQ7qRAJgXoFYaG7YY7N7%2BLkPJhevA1Wy1wdeH%2FIw%3D%3D'

In [2]:
#위도 =111.195km , 경도 88.8km , 2km 박스일 때 반경  1.414
def dis_to_lat(km):
    return km/111.195
def dis_to_lon(km):
    return km/88.8

def lat_to_dis(lat):
    return lat*111.195
def lon_to_dis(lon):
    return lon*88.8

def get_xy(data,lon, lat,km):
    lat_size = dis_to_lat(km)
    lon_size = dis_to_lon(km)
    return data[(data['위도']<=lat+lat_size/2) & (data['위도']>=lat-lat_size/2) &
                 (data['경도']<=lon+lon_size/2) & (data['경도']>=lon-lon_size/2)]

In [3]:
# park
def get_park(lon,lat,km):
    park_seoul = pd.read_csv('./Seoul_park.csv',encoding='CP949')
    park_seoul = park_seoul[['공원명','Y좌표(WGS84)','X좌표(WGS84)']]
    park_seoul=park_seoul.rename({'Y좌표(WGS84)' : '위도','X좌표(WGS84)': '경도'},axis=1)
    return get_xy(park_seoul, lon, lat, km)

#school
def get_school(lon,lat,km):
    school_data = pd.read_csv('./ele_middle_school.csv',encoding='CP949')
    seoul_school = school_data[school_data['시도교육청명']=='서울특별시교육청']
    seoul_school = seoul_school[['학교명','학교급구분','위도','경도']]
    return get_xy(seoul_school,lon,lat,km)

# commer
key_commer = 'bG%2BZ%2BmDdiTVy%2Faq2OLF%2FKCockUZQnuHoXbUTjrFJYWbe5ZtQ7qRAJgXoFYaG7YY7N7%2BLkPJhevA1Wy1wdeH%2FIw%3D%3D'
def get_commer(api_key, cx, cy, km):
    pageNo = 1
    commercial_info = []
    col = ''
    radius = (km/2)*1414
    while True:
        url = 'http://apis.data.go.kr/B553077/api/open/sdsc/storeListInRadius?radius=%s&cx=%s&cy=%s&serviceKey=%s&pageNo=%s'%(radius, cx, cy, api_key, pageNo)
        response = requests.get(url)
        result = response.text
        soup = BeautifulSoup(result,'lxml')
        col = soup.columns.text.split(',')
        
        if soup.resultcode.text == '00':
            for item in soup.find_all('item'):
                commercial_info.append(item.text.split('\n')[1:][:-1])
            pageNo += 1
        else:
            break
    
    if len(commercial_info) > 0:
        commercial_info = pd.DataFrame(commercial_info, columns=col)
    else:
        commercial_info = pd.DataFrame()
    commercial_info = commercial_info.astype({'경도':'float','위도':'float'})
    commercial_info = get_xy(commercial_info, cx, cy, km/2)  
    return commercial_info

def get_fire(lon,lat,km):
    fire = pd.read_csv('./fire_station.csv',encoding='CP949')
    fire = fire.astype({'경도':'float','위도':'float'})
    get_xy(fire,lon,lat,2)    

MT1	대형마트  
CS2	편의점  
PS3	어린이집, 유치원  
SC4	학교  
AC5	학원  
PK6	주차장  
OL7	주유소, 충전소  
SW8	지하철역  
BK9	은행  
CT1	문화시설  
AG2	중개업소  
PO3	공공기관  
AT4	관광명소  
AD5	숙박  
FD6	음식점  
CE7	카페  
HP8	병원  
PM9	약국

In [104]:
def search_keyword(key,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
    headers = {"Authorization": "KakaoAK {}".format(key)}
    URL_key = "https://dapi.kakao.com/v2/local/search/keyword.json"
    """
    05 키워드 검색
    """
    params = {"query": f"{query}"}

    if category_group_code != None:
        params['category_group_code'] = f"{category_group_code}"
    if x != None:
        params['x'] = f"{x}"
    if y != None:
        params['y'] = f"{y}"
    if radius != None:
        params['radius'] = f"{radius}"
    if rect != None:
        params['rect'] = f"{rect}"
    if page != None:
        params['page'] = f"{page}"
    if size != None:
        params['size'] = f"{params}"
    if sort != None:
        params['sort'] = f"{sort}"

    res = requests.get(URL_key, headers=headers, params=params)
    document = json.loads(res.text)

    return document


def search_category(key,category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
    headers = {"Authorization": "KakaoAK {}".format(key)}
    URL_cat = "https://dapi.kakao.com/v2/local/search/category.json"
    """
    06 카테고리 검색
    """
    params = {'category_group_code': f"{category_group_code}",
              'x': f"{x}",
              'y': f"{y}"}

    if radius != None:
        params['radius'] = f"{radius}"
    if rect != None:
        params['rect'] = f"{rect}"
    if page != None:
        params['page'] = f"{page}"
    if size != None:
        params['size'] = f"{size}"
    if sort != None:
        params['sort'] = f"{sort}"

    res = requests.get(URL_cat, headers=headers, params=params)
    document = json.loads(res.text)

    return document



def get_public(cat,lon,lat):
    radius = 1415
    col = ['category_group_name','place_name', 'x','y']
    result = search_category(rest_api_key,cat, lon, lat, radius)
    result = pd.DataFrame(result['documents'])
    result = result[col]
    result = result.rename(columns = {'x': '경도','y':'위도'})
    result = result.astype({'경도':'float','위도':'float'})
    return result

get_public('SC4',lon,lat)

,category_group_name,place_name,경도,위도
0,학교,상명대학교 서울캠퍼스,126.955350,37.602322
1,학교,서울세검정초등학교,126.961170,37.603096
2,학교,상명대학교사범대학 부속여자고등학교,126.953823,37.603306
3,학교,상명대학교사범대학 부속초등학교,126.956411,37.600784
4,학교,서울홍은초등학교,126.949469,37.597555
5,학교,서울예술고등학교,126.968902,37.605547
6,학교,인왕중학교,126.951778,37.592723
7,학교,상명대학교 사범대학부속여자중학교,126.954158,37.602819


In [62]:
lon =126.955009
lat =37.6027953
km = 2

park = get_park(lon,lat,km)
school = get_school(lon,lat,km)
#commer = get_commer(key_commer,lon,lat,km)
fire = get_fire(lon, lat, km)
public = get_public(lon,lat)

# School_data  
학교 데이터를 추출

In [4]:
school_data = pd.read_csv('./ele_middle_school.csv',encoding='CP949')

In [5]:
seoul_school = school_data[school_data['시도교육청명']=='서울특별시교육청']
seoul_school = seoul_school[['학교명','학교급구분','위도','경도']]

In [6]:
seoul_school.head(3)

,학교명,학교급구분,위도,경도
4,안천중학교,중학교,37.459524,126.887499
8,서울잠일초등학교,초등학교,37.514742,127.081636
12,홍익디자인고등학교,고등학교,37.563511,126.919166


위도 경도 범위 내의 학교 추출

In [8]:
tmp

,학교명,학교급구분,위도,경도
318,상명대학교사범대학부속여자중학교,중학교,37.604109,126.955159
319,상명대학교사범대학부속초등학교,초등학교,37.600441,126.956715
436,상명대학교사범대학부속여자고등학교,고등학교,37.604109,126.955159


# 상권 가져오기

In [9]:
import requests


key_commer = 'bG%2BZ%2BmDdiTVy%2Faq2OLF%2FKCockUZQnuHoXbUTjrFJYWbe5ZtQ7qRAJgXoFYaG7YY7N7%2BLkPJhevA1Wy1wdeH%2FIw%3D%3D'

In [7]:
def get_commercial_info_by_radius(api_key, cx, cy, km):
    pageNo = 1
    commercial_info = []
    col = ''
    radius = (km/2)*1414
    while True:
        url = 'http://apis.data.go.kr/B553077/api/open/sdsc/storeListInRadius?radius=%s&cx=%s&cy=%s&serviceKey=%s&pageNo=%s'%(radius, cx, cy, api_key, pageNo)
        response = requests.get(url)
        result = response.text
        soup = BeautifulSoup(result,'lxml')
        col = soup.columns.text.split(',')
        
        if soup.resultcode.text == '00':
            for item in soup.find_all('item'):
                commercial_info.append(item.text.split('\n')[1:][:-1])
            pageNo += 1
        else:
            break
    
    if len(commercial_info) > 0:
        commercial_info = pd.DataFrame(commercial_info, columns=col)
    else:
        commercial_info = pd.DataFrame()
    commercial_info = commercial_info.astype({'경도':'float','위도':'float'})
    commercial_info = get_xy(commercial_info, cx, cy, km/2)  
    return commercial_info
box_size = 2 # 2km
commercial_info_by_radius = get_commercial_info_by_radius(key_commer,lon ,lat, box_size)
commercial_info_by_radius

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
12,11780029,다향,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111018200100880049021717,,서울특별시 종로구 진흥로 455,110803,03007,,2,,126.958884,37.607174
13,11702191,옛날민속집1호점,1호점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111018200100640005023458,,서울특별시 종로구 진흥로 469,110803,03007,,1,,126.960343,37.606520
24,11911333,현미세탁소,,F,생활서비스,F02,세탁/가사서비스,F02A01,세탁소/빨래방,S96912,...,1141011800100090428012055,,서울특별시 서대문구 포방터길 68,120100,03606,,1,,126.949465,37.599657
25,11888257,수해유통,,F,생활서비스,F14,자동차/이륜차,F14A01,자동차정비/카센타,,...,1141011800104540000010868,극동아파트,서울특별시 서대문구 포방터10길 33,120773,03601,,2,213,126.951374,37.602449
28,11912099,이이일공공,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1111018500100460001023945,,서울특별시 종로구 홍지문길 7,110020,03015,,1,,126.956808,37.600137
30,11898053,새나라오토바이상사,,F,생활서비스,F14,자동차/이륜차,F14A03,오토바이판매/수리,,...,1111018500101220023025095,,서울특별시 종로구 세검정로 220,110020,03021,,1,,126.957860,37.599380
31,11910590,구기촌,,Q,음식,Q04,분식,Q04A01,라면김밥분식,I56194,...,1111018200100580003022094,,서울특별시 종로구 진흥로 462,110803,03013,,1,,126.959174,37.606681
47,11951303,카페카자드지나,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1111018200100950011031984,,서울특별시 종로구 진흥로 450,110803,03012,,1,,126.958056,37.607257
48,11930100,명품공인중개사사무소,,L,부동산,L01,부동산중개,L01A01,부동산중개,L68221,...,1111018200100580004022352,,서울특별시 종로구 진흥로 460,110803,03013,,2,,126.959049,37.606734
69,11984144,쥬시상명대점,상명대점,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1111018500100550001023918,,서울특별시 종로구 홍지문2길 7,110020,03015,,2,,126.955913,37.600976


In [8]:
commercial_info_by_radius['상권업종대분류명'].unique()

array(['음식', '생활서비스', '부동산', '소매', '관광/여가/오락', '학문/교육', '숙박'],
      dtype=object)

In [12]:
tmp =commercial_info_by_radius.astype({'경도':'float','위도':'float'})

In [19]:
s =get_xy(tmp,lon,lat,2)

In [20]:
s

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,10976300,꾸러기어린이집,,R,학문/교육,R08,유아교육,R08A02,어린이집,Q87210,...,1141011800104550000013034,벽산아파트,서울특별시 서대문구 세검정로1길 95,120771,03612,103,,103,126.945666,37.595208
1,11748330,홍은손칼국수,,Q,음식,Q04,분식,Q04A02,국수/만두/칼국수,I56194,...,1141011800100480100013158,,서울특별시 서대문구 홍은중앙로 27,120100,03616,,1,,126.945955,37.593977
3,10870518,세븐일레븐,홍은중앙길점,D,소매,D03,종합소매점,D03A01,편의점,G47122,...,1141011800100090106012274,,서울특별시 서대문구 홍은중앙로 96,120100,03605,,,,126.948056,37.599158
4,11751065,교촌치킨,홍은1호,Q,음식,Q05,닭/오리요리,Q05A08,후라이드/양념치킨,I56193,...,1141011100102790053014396,예학빌딩,서울특별시 서대문구 세검정로 89,120090,03619,,1,,126.948636,37.595757
5,11770135,카페루,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1141011100102860001014178,,서울특별시 서대문구 포방터길 27,120090,03617,,1,,126.948742,37.597948
6,11769948,세븐일레븐,홍제포방교점,D,소매,D03,종합소매점,D03A01,편의점,G47122,...,1141011100102850033015817,,서울특별시 서대문구 포방터길 20,120090,03617,,1,,126.948736,37.597569
7,11779396,마리내,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1141011100102860007014226,,서울특별시 서대문구 간호대로2길 22,120090,03617,,1,,126.948997,37.597500
8,11853192,케이와이영재토론수학학원,,R,학문/교육,R01,학원-보습교습입시,R01A01,학원-입시,P85501,...,1141011100102790027032988,,서울특별시 서대문구 세검정로 92,120090,03618,,2,,126.949056,37.595707
9,11851740,마을사랑방솜씨,,Q,음식,Q12,커피점/카페,Q12A01,커피전문점/카페/다방,I56220,...,1141011100102850041014100,,서울특별시 서대문구 간호대로2길 10,120090,03617,,1,,126.949140,37.596960
10,11784291,곰닭,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1141011800100090044012706,,서울특별시 서대문구 홍은중앙로 116-1,120100,03605,,1,,126.949131,37.600880


# 병의원 찾기

# 주변 공원

In [7]:
park_seoul = pd.read_csv('./Seoul_park.csv',encoding='CP949')

In [9]:
park_seoul['공원명'].unique()

array(['남산도시자연공원', '길동생태공원', '서울대공원', '서울숲', '월드컵공원', '광화문시민열린마당',
       '진관근린공원(구파발폭포)', '중마루근린공원', '응봉공원', '매화근린공원', '성수근린공원', '샛마을근린공원',
       '여의도근린공원', '간데메근린공원', '훈련원근린공원', '영등포근린공원', '보라매근린공원', '어린이대공원',
       '천호근린공원', '용산가족공원', '독립근린공원', '낙산근린공원', '양재시민의숲', '관악산도시자연공원',
       '효창근린공원', '감로천생태공원(관악산)', '개운산근린공원', '개웅산근린공원', '경희궁', '국립현충원',
       '계남근린공원', '고척근린공원', '파리근린공원', '허준공원', '궁동근린공원', '궁산근린공원',
       '까치산근린공원', '꿩고개근린공원', '노량진공원', '답십리근린공원', '대모산도시자연공원', '탑골근린공원',
       '갈산근린공원', '천마근린공원', '도곡근린공원', '도산근린공원', '청량공원', '초안산근린공원',
       '동명근린공원', '마로니에공원', '자매근린공원', '장지근린공원', '명일근린공원', '방화근린공원',
       '아시아근린공원', '배봉산근린공원', '안산도시자연공원', '염창근린공원', '백련근린공원', '오금근린공원',
       '오동근린공원', '봉산도시자연공원', '온수도시자연공원', '봉은공원', '올림픽공원', '와룡근린공원',
       '봉화산근린공원', '북악산도시자연공원', '북한산국립공원', '외국인묘지공원', '불광근린공원',
       '불암산도시자연공원', '용마폭포공원', '사육신공원', '용왕산근린공원', '사직근린공원', '우장산근린공원',
       '삼일근린공원', '삼청근린공원', '인왕산도시자연공원', '상도근린공원', '일자산도시자연공원', '쌍문근린공원',
       '수락산도시자연공원', '송파나루근린공원(

In [43]:
park_seoul=park_seoul.rename({'Y좌표(WGS84)' : '위도','X좌표(WGS84)': '경도'},axis=1)

In [45]:
tmp = get_xy(park_seoul, lon, lat, 2)

In [46]:
tmp

,공원명,위도,경도


# 소방서

In [5]:
def get_park(lon,lat,km):
    park_seoul = pd.read_csv('./Seoul_park.csv',encoding='CP949')
    park_seoul = park_seoul[['공원명','Y좌표(WGS84)','X좌표(WGS84)']]
    park_seoul=park_seoul.rename({'Y좌표(WGS84)' : '위도','X좌표(WGS84)': '경도'},axis=1)
    return get_xy(park_seoul, lon, lat, km)

In [6]:
fire = pd.read_csv('./fire_station.csv',encoding='CP949')

In [9]:
def get_fire(lon,lat,km):
    fire = pd.read_csv('./fire_station.csv',encoding='CP949')
    fire = fire.astype({'경도':'float','위도':'float'})
    get_xy(fire,lon,lat,2)

,순번,서소코드,방면코드,중앙점X좌표,중앙점Y좌표,서소이름,위도,경도
4,5,71236,1,197056.96972,455503.4815,종로소방서,37.599100,126.986175
5,6,78236,1,193175.04150,457560.4345,은평소방서,37.617611,126.922713
10,11,83236,1,191941.16600,451421.9075,마포소방서,37.562293,126.908779
19,20,92236,1,194316.92450,453703.0735,서대문소방서,37.582869,126.935668
22,23,72236,1,199487.02850,450935.4220,중부소방서,37.557945,126.994195


# 경찰서

In [10]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
import requests
from urllib.parse import urlparse
import pandas as pd
import folium

MT1	대형마트  
CS2	편의점  
PS3	어린이집, 유치원  
SC4	학교  
AC5	학원  
PK6	주차장  
OL7	주유소, 충전소  
SW8	지하철역  
BK9	은행  
CT1	문화시설  
AG2	중개업소  
PO3	공공기관  
AT4	관광명소  
AD5	숙박  
FD6	음식점  
CE7	카페  
HP8	병원  
PM9	약국

In [48]:
rest_api_key = 'c09c78655acb25593e9c6f07b0f42f90'

In [65]:
import json
import requests

rest_api_key = 'c09c78655acb25593e9c6f07b0f42f90'

def search_keyword(key,query,category_group_code=None,x=None,y=None,radius=None,rect=None,page=None,size=None,sort=None):
    headers = {"Authorization": "KakaoAK {}".format(key)}
    URL_key = "https://dapi.kakao.com/v2/local/search/keyword.json"
    """
    05 키워드 검색
    """
    params = {"query": f"{query}"}

    if category_group_code != None:
        params['category_group_code'] = f"{category_group_code}"
    if x != None:
        params['x'] = f"{x}"
    if y != None:
        params['y'] = f"{y}"
    if radius != None:
        params['radius'] = f"{radius}"
    if rect != None:
        params['rect'] = f"{rect}"
    if page != None:
        params['page'] = f"{page}"
    if size != None:
        params['size'] = f"{params}"
    if sort != None:
        params['sort'] = f"{sort}"

    res = requests.get(URL_key, headers=headers, params=params)
    document = json.loads(res.text)

    return document


def search_category(key,category_group_code, x, y, radius=None, rect=None, page=None, size=None, sort=None):
    headers = {"Authorization": "KakaoAK {}".format(key)}
    URL_cat = "https://dapi.kakao.com/v2/local/search/category.json"
    """
    06 카테고리 검색
    """
    params = {'category_group_code': f"{category_group_code}",
              'x': f"{x}",
              'y': f"{y}"}

    if radius != None:
        params['radius'] = f"{radius}"
    if rect != None:
        params['rect'] = f"{rect}"
    if page != None:
        params['page'] = f"{page}"
    if size != None:
        params['size'] = f"{size}"
    if sort != None:
        params['sort'] = f"{sort}"

    res = requests.get(URL_cat, headers=headers, params=params)
    document = json.loads(res.text)

    return document



def get_public(lon,lat):
    radius = 1415
    col = ['category_group_name','place_name', 'x','y']
    result = search_category(rest_api_key,category_group_code, lon, lat, radius)
    result = pd.DataFrame(result['documents'])
    result = result[col]
    result = result.rename(columns = {'x': '경도','y':'위도'})
    result = result.astype({'경도':'float','위도':'float'})
    return result

get_public(lon,lat)

In [85]:
category_group_code = "PO3"
# x = 127.02449138906029
# y = 37.50229485705552
radius = 1415
result_6 = search_category(rest_api_key,category_group_code, lon, lat, radius)

In [ ]:
# address_name': '서울 서대문구 홍은동 11-853',
#   'category_group_code': 'PO3',
#   'category_group_name': '공공기관',
#   'category_name': '사회,공공기관 > 지방행정기관 > 행정복지센터 > 주민센터',
#   'distance': '819',
#   'id': '10971625',
#   'phone': '02-330-8129',
#   'place_name': '홍은1동주민센터',
#   'place_url': 'http://place.map.kakao.com/10971625',
#   'road_address_name': '서울 서대문구 홍은중앙로 85',
#   'x': '126.947108251211',
#   'y': '37.5989166778989

In [103]:
def get_public(lon,lat):
    radius = 1415
    col = ['category_group_name','place_name', 'x','y']
    result = search_category(rest_api_key,category_group_code, lon, lat, radius)
    result = pd.DataFrame(result['documents'])
    result = result[col]
    result = result.rename(columns = {'x': '경도','y':'위도'})
    result = result.astype({'경도':'float','위도':'float'})
    return result

get_public(lon,lat)

,category_group_name,place_name,경도,위도
0,공공기관,홍은1동주민센터,126.947108,37.598917
1,공공기관,세검정파출소,126.958463,37.600024
2,공공기관,평창동주민센터,126.968358,37.606392
3,공공기관,신영119안전센터,126.960839,37.605743
4,공공기관,홍제3동주민센터,126.949743,37.593782
5,공공기관,구기치안센터,126.956397,37.609513
6,공공기관,부암동주민센터,126.964061,37.592414
7,공공기관,홍은119안전센터,126.947072,37.598729
8,공공기관,평창파출소,126.966632,37.605270
9,공공기관,홍은파출소,126.946382,37.595258
